In [66]:
import numpy as np

In [67]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  #   웨이브 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + 웨이브 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # + 잡음
    return series[..., np.newaxis].astype(np.float32)

batch_size 개수만큼의 시계열 데이터가 생성

각 시계열 데이터는 n_steps 만큼의 데이터가 있고,

앞선 n_steps - 1 개의 데이터를 통해 n_steps 번째의 데이터를 예측한다.

In [68]:
np.random.seed(42)

n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [69]:
X_train.shape

(7000, 50, 1)

In [70]:
X_train[0].shape

(50, 1)

In [71]:
y_train.shape

(7000, 1)

In [72]:
# 활성화 함수
class activation_function:
  # 시그모이드 함수
  def sigmoid(self, x):
    return 1 / (1+np.e**-x)
  
  # 시그모이드 함수의 미분 함수
  def sigmoid_diff(self, x):
    return x * (1 - x)

  # 값을 그대로 출력하는 함수
  def non(self, x):
    return x
  
  def non_diff(self, x):
    return 1

In [73]:
# 비용 함수 클래스, 비용 함수의 종류에 따라 다른 클래스 내 함수를 사용한다.
class cost_function:
  # 예측값
  predict = []
  # 타겟값
  target = []
  # 비용 함수값
  error_cost = []

  # 오차 제곱합
  def errer_squared_sum(self, predict, target):
    self.predict = predict
    self.target = target

    self.error_cost = np.sum(0.5*((predict - target)**2))
    return self.error_cost

  # 오차 제곱합 미분 함수
  def diff_error_squared_sum(self):
    return (self.predict[0][0] - self.target[0][0])

In [74]:
class RNN_vector_to_sequence:
    #입력값
    input_data = []
    
    #가중치
    weight = []
    
    #편향값
    bias = []

    # 입력 가중치
    input_w = []

    # 이전 출력에 대한 가중치
    before_w = []

    # 편향 값 bais
    bias = []

    # 노드 입력
    node_input = []

    # 노드 출력
    node_output = []

    # 타겟, 목푯값
    target = []

    # 활성화 함수
    activation = activation_function()

    # 비용 함수
    cost = cost_function()

    # 예측 결과
    predict = []

    # 비용 함수값
    cost_reuslt = []

    # 각 층의 델타 값의 저장
    delta = []

     # 각 가중치 업데이트 크기
    input_weight_update = []

    before_weight_update = []

    bias_update = []

    #초기 학습, 임의의 가중치가 생성된다.
    def delta_to_sequence_cal_rnn_test(self, input, target):

      # 타겟값의 저장
      self.target = target

      # 입력 데이터의 저장
      self.input = input

      # 입력 데이터 크기에 맞는 가중치 값이 생성되어야 한다.
      # 입력 (3,1) 데이터에 대한 연산 결과 (1,1)를 예측한다고 가정,

      # 입력값에 사용되는 가중치
      # (3,1) 크기의 입력 데이터에 대해 (1,1) 의 행렬곱 연산 결과를 얻기 위해, (1,3) 크기의 가중치 행렬이 필요하다
      self.input_w = np.random.rand(target.shape[0], input.shape[1])
      
      # 이전 출력에 대한 가중치
      # (1,1) 노드 출력에 대해 (1,1) 의 행렬곱 연산 결과를 얻기 위한, (1,1) 크기의 가중치 행렬
      self.before_w = np.random.rand(target.shape[0], target.shape[0])

      # bias 값 생성, (1,1) 행렬에 더해지기 위한 (1,1) 크기의 bias 값 생성
      self.bias = np.random.rand(target.shape[0], target.shape[0])

      # 이전 노드의 출력, 첫 부분의 경우 이 값이 0이다. 행렬의 크기는 (1,1)
      before = np.zeros((target.shape[0], target.shape[0]))

      # (3, n) 크기의 입력 데이터에서 n 번의 반복을 수행하게 됨
      for i in range(input.shape[0]):
        # 노드 입력 데이터의 numpy 행렬 생성
        input_data = input[i].reshape(input.shape[1],-1)

        # RNN 노드 입력 값 계산, 이전 노드 출력 @ 가중치 + 입력 데이터 @ 가중치
        node_input = (self.before_w @ before) + (self.input_w @ input_data) + self.bias
        self.node_input.append(node_input)

        # 노드 출력 계산, 활성화 함수 연산을 수행한다.
        node_output = self.activation.non(node_input)
        self.node_output.append(node_output)

        # 타입 스텝의 출력이 다음 노드의 입력이 된다.
        before = node_output

      self.predict = node_output

      self.cost_result = self.cost_cal()

      return self.predict, self.cost_result

    def cost_cal(self):
      return self.cost.errer_squared_sum(self.predict, self.target)

    def update_weight(self, learning_rate):
      # 원활한 연산을 위해 마지막 노드부터 거꾸로 계산된 delta 값을 뒤집어준다.
      self.delta = self.delta[::-1]

      result_input = 0

      result_before = 0

      result_bias = 0

      # 해당 delta 값과의 데이터 입력값을 통해 input_w 의 가중치 변화량을 계산할 수 있다
      for i in range(self.input.shape[0]):
        result_input = result_input + self.delta[i] * self.node_input[i]

        result_before = result_before + self.delta[i] * self.input[i]

        result_bias = result_bias + self.delta[i]

      self.input_weight_update = result_input
      
      self.before_weight_update = result_before

      self.bias_update = result_bias

      # 가중치 업데이트
      self.input_w = self.input_w - (self.input_weight_update * learning_rate)

      self.before_w = self.before_w - (self.before_weight_update * learning_rate)

      self.bias = self.bias - (self.bias_update * learning_rate)

    def iterations(self, iterations, learning_rate):
      # j 번의 학습
      for j in range(iterations):
        # 초기화
        self.node_input = []
        self.node_output = []
        self.delta = []

        # 이전 노드의 출력, 첫 부분의 경우 이 값이 0이다. 행렬의 크기는 (1,1)
        before = np.zeros((self.target.shape[0], self.target.shape[0]))

        for i in range(input.shape[0]):
          # 노드 입력 데이터의 numpy 행렬 생성
          input_data = input[i].reshape(input.shape[1],-1)

          # RNN 노드 입력 값 계산, 이전 노드 출력 @ 가중치 + 입력 데이터 @ 가중치 + bias
          # 업데이트된 가중치, 편향값으로 계산이 이뤄진다.
          node_input = (self.before_w @ before) + (self.input_w @ input_data) + self.bias
          self.node_input.append(node_input)

          # 노드 출력 계산, 활성화 함수 연산을 수행한다.
          node_output = self.activation.non(node_input)
          self.node_output.append(node_output)

          # 순환 노드의 다음 노드 입력값
          before = node_output

        # 매 학습마다 예측값이 달라짐
        self.predict = before

        # 예측값에 따른 비용 함숫값 계산
        self.cost_result = self.cost_cal()

        print(self.cost_cal())

        # 각 노드별 비용 함수의 변화량 계산
        self.cal_delta()

        # 계산한 delta 값을 통해 가중치 값 갱신
        self.update_weight(learning_rate)

    def cal_delta(self):
      self.delta = []
      #마지막 노드의 변화량에 대한 오차 함수의 변화량 계산
      #delta = (self.cost.diff_error_squared_sum() * self.activation.non_diff(self.predict))
      delta = (self.cost.diff_error_squared_sum() * self.activation.non_diff(self.node_output[len(self.node_output) - 1]))
      self.delta.append(delta)

      # n-1 개의 delta 값을 계산하기 위한 반복
      for i in range(input.shape[0] - 1, 0, -1):
        # 노드 출력의 변화에 대한 비용 함수의 변화량 계산
        delta = self.before_w * delta

        # 활성화 함수에 따른 변화량 계산, 해당 값이 노드 변화에 대한 비용 함수의 변화량이 된다.
        delta = delta * self.activation.non_diff(self.node_output[i])

        # 노드별 delta 값의 저장
        self.delta.append(delta)



In [75]:
rnn = RNN_vector_to_sequence()

In [76]:
input = X_train[0]

In [77]:
target = y_train[0].reshape(1,-1)

In [78]:
target

array([[-0.35489398]], dtype=float32)

In [79]:
rnn.delta_to_sequence_cal_rnn_test(input, target)

(array([[1.57795929]]), 1.867960879028231)

In [80]:
print(rnn.input_w, rnn.before_w, rnn.bias)

[[0.53680354]] [[0.45702764]] [[0.84214604]]


In [81]:
rnn.cal_delta()

In [82]:
rnn.update_weight(0.01)

In [83]:
print(rnn.input_weight_update, rnn.before_weight_update, rnn.bias_update)

[[6.0962773]] [[0.09709869]] [[3.55976362]]


In [84]:
rnn.iterations(100, 0.01)

1.73179085461459
1.6067607711006486
1.49179330634851
1.3859389478963788
1.2883573833162305
1.198302120081547
1.1151076928027817
1.038178947039736
0.9669820247324957
0.9010367256875604
0.8399100065512232
0.7832104235703583
0.7305833546236293
0.6817068756371395
0.6362881889148133
0.5940605132992136
0.5547803702630245
0.5182252039916695
0.48419129034720704
0.4524918908767738
0.4229556207044987
0.3954250000775542
0.36975516574434775
0.3458127213748758
0.32347471041387005
0.30262769482617835
0.28316692809684346
0.2649956108580792
0.24802421975604574
0.23216990093474452
0.21735592100392923
0.2035111696176989
0.19056970758403433
0.17847035582866527
0.16715632119660784
0.15657485511102448
0.14667694181537363
0.13741701340256135
0.12875268890809138
0.12064453517240625
0.11305584751716956
0.1059524482709149
0.09930250158707764
0.09307634306348303
0.08724632283616017
0.08178666096422824
0.07667331404234747
0.07188385207015789
0.06739734471338833
0.06319425616394837
0.05925634789568706
0.055566588

In [86]:
print(rnn.predict, rnn.target)

[[-0.28241821]] [[-0.35489398]]
